In [47]:
import numpy as np
import pandas as pd
Data_train = pd.read_csv("train.csv")
Data_test  = pd.read_csv("test.csv")

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

X_train, X_test, y_train, y_test = train_test_split(Data_train.iloc[:,1:6], Data_train.iloc[:,6],random_state = 0, stratify = Data_train.iloc[:,6])


X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, random_state = 0, stratify = y_train)


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(decode_error="ignore",ngram_range=(1, 2))


In [21]:
X_train_train_title_bigram = vectorizer.fit_transform(X_train_train.title)
X_valid_title_bigram  = vectorizer.transform(X_valid.title)
X_train_train_abstract_bigram = vectorizer.fit_transform(X_train_train.abstract)
X_valid_abstract_bigram  = vectorizer.transform(X_valid.abstract)
X_train_train.references = X_train_train.references.fillna("")
X_valid.references = X_valid.references.fillna("")
vectorizer_ref = CountVectorizer(decode_error="ignore",ngram_range=(1,1))
X_train_train_references_bigram = vectorizer_ref.fit_transform(X_train_train.references)
X_valid_references_bigram  = vectorizer_ref.transform(X_valid.references)

from scipy.sparse import coo_matrix, hstack
X_train_train_bigram_numeric = hstack([X_train_train_title_bigram, X_train_train_abstract_bigram, X_train_train_references_bigram, X_train_train.iloc[:, [2,4]]])

X_valid_bigram_numeric  = hstack([X_valid_title_bigram, X_valid_abstract_bigram, X_valid_references_bigram, X_valid.iloc[:, [2,4]]])

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

scores = []
n_estimator_list = [300, 400, 500, 600, 1000]
for this_n_estimators in n_estimator_list:
    rf = RandomForestClassifier(n_jobs=-1, n_estimators = this_n_estimators)
    rf.fit(X_train_train_bigram_numeric, y_train_train)
    scores.append(rf.score(X_valid_bigram_numeric, y_valid))
    

print("The scores is {}".format(scores))

The scores is [0.8359277151547683, 0.8316335659330828, 0.8352120236178208, 0.838074789765611, 0.8378958668813741]


In [32]:
X_train_title_bigram = vectorizer.fit_transform(X_train.title)
X_test_title_bigram  = vectorizer.transform(X_test.title)
X_train_abstract_bigram = vectorizer.fit_transform(X_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(X_test.abstract)
X_train.references = X_train.references.fillna("")
X_test.references = X_test.references.fillna("")
X_train_references_bigram = vectorizer_ref.fit_transform(X_train.references)
X_test_references_bigram  = vectorizer_ref.transform(X_test.references)
X_train_bigram_numeric = hstack([X_train_title_bigram, X_train_abstract_bigram, X_train_references_bigram, X_train.iloc[:, [2,4]]])
X_test_bigram_numeric  = hstack([X_test_title_bigram, X_test_abstract_bigram, X_test_references_bigram, X_test.iloc[:, [2,4]]])


In [ ]:
bestID = np.argmax(scores)
rf = RandomForestClassifier(n_jobs=-1, n_estimators = n_estimator_list[bestID])
rf.fit(X_train_bigram_numeric, y_train)
test_score = rf.score(X_test_bigram_numeric, y_test)
print("Test score is {}".format(test_score))



In [48]:
# from adspy_shared_utilities import plot_feature_importances
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,4), dpi=80)
# plot_feature_importances(rf, X_train_bigram_numeric.col)
# plt.show()

print('Feature importances: {}'.format(rf.feature_importances_))


Feature importances: [0.33978048 0.66021952]


In [42]:
bestID = np.argmax(scores)
rf = RandomForestClassifier(n_jobs=-1, n_estimators = n_estimator_list[bestID])
rf.fit(X_train.iloc[:,[2,4]], y_train)
test_score = rf.score(X_test.iloc[:,[2,4]], y_test)
print("Test score is {}".format(test_score))

Test score is 0.34174158057158194


In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(decode_error="ignore",ngram_range=(1, 2))
X_train_title_bigram = vectorizer.fit_transform(X_train.title)
X_test_title_bigram  = vectorizer.transform(X_test.title)
X_train_abstract_bigram = vectorizer.fit_transform(X_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(X_test.abstract)
X_train.references = X_train.references.fillna("")
X_test.references = X_test.references.fillna("")
X_train_references_bigram = vectorizer.fit_transform(X_train.references)
X_test_references_bigram  = vectorizer.transform(X_test.references)
from scipy.sparse import coo_matrix, hstack
X_train_bigram_numeric = hstack([X_train_title_bigram, X_train_abstract_bigram, X_train_references_bigram, X_train.iloc[:, [2,4]]])
X_test_bigram_numeric  = hstack([X_test_title_bigram, X_test_abstract_bigram, X_test_references_bigram, X_test.iloc[:, [2,4]]])

In [ ]:
test_scores = []
train_scores = []
C_list = np.linspace(1,100, 20)
for this_C in C_list:
    lr = LogisticRegression(C= this_C, penalty = 'elasticnet', random_state=0, solver = "saga", n_jobs=-1,  l1_ratio = 0.5)
    lr.fit(X_train_bigram_numeric, y_train)
    tr = lr.score(X_train_bigram_numeric, y_train)
    te = lr.score(X_test_bigram_numeric, y_test)
    print("C = {},Training score is {}, testing score is {}\n".format(this_C, tr, te))
    train_scores.append(tr)
    test_scores.append(te)


In [110]:
test_scores = []
train_scores = []
C_list = np.linspace(0.0001, 0.05, 20)
for this_C in C_list:
    lr = LogisticRegression(C= this_C, penalty = 'l2', random_state=0)
    lr.fit(X_train_bigram_numeric, y_train)
    tr = lr.score(X_train_bigram_numeric, y_train)
    te = lr.score(X_test_bigram_numeric, y_test)
    print("C = {},Training score is {}, testing score is {}\n".format(this_C, tr, te))
    train_scores.append(tr)
    test_scores.append(te)


C = 0.0001,Training score is 0.7453032742887815, testing score is 0.7304441164631692

C = 0.002726315789473684,Training score is 0.9096439434603686, testing score is 0.8561653025627264

C = 0.005352631578947368,Training score is 0.9356324923957774, testing score is 0.8647524486783845

C = 0.00797894736842105,Training score is 0.9515118983718017, testing score is 0.8690460217362136

C = 0.010605263157894736,Training score is 0.9634550008946144, testing score is 0.8709244599490138

C = 0.01323157894736842,Training score is 0.9724458758275183, testing score is 0.8709244599490138

C = 0.015857894736842102,Training score is 0.9771426015387368, testing score is 0.871863679055414

C = 0.018484210526315787,Training score is 0.982778672392199, testing score is 0.8737421172682142

C = 0.021110526315789472,Training score is 0.9851046698872786, testing score is 0.8744129880585

C = 0.023736842105263157,Training score is 0.9882358203614242, testing score is 0.8746813363746142

C = 0.026363157894736

In [113]:
[i - j for i,j in zip(train_scores, test_scores)]

[0.014859157825612312,
 0.053478640897642205,
 0.07088004371739287,
 0.08246587663558813,
 0.09253054094560065,
 0.1015214158785045,
 0.10527892248332282,
 0.1090365551239848,
 0.11069168182877864,
 0.11355448398681,
 0.11476221345540905,
 0.11614900984920906,
 0.11775940583294364,
 0.11798264532046798,
 0.11892227854463988,
 0.11999576183469973,
 0.1208904662814867,
 0.12089070034805338,
 0.12106964123741082,
 0.12057749527503625]

In [61]:
rf = RandomForestClassifier(n_jobs=-1, max_features=0.000002, n_estimators=500)
rf.fit(X_train_bigram_numeric, y_train)
rf.feature_impotance

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=2e-06, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [80]:
fI = rf.feature_importances_


In [82]:
fI = pd.DataFrame(fI)
fI['INDEX'] = fI.index

In [84]:
selected_feature_index = fI.sort_values(by = [0],ascending = False).INDEX

In [88]:
help(coo_matrix.trunc)

Help on function trunc in module scipy.sparse.data:

trunc(self)
    Element-wise trunc.
    
    See numpy.trunc for more information.



In [90]:
X_train_bigram_numeric_csr = X_train_bigram_numeric.tocsr()
X_test_bigram_numeric_csr = X_test_bigram_numeric.tocsr()

In [107]:
X_train_bigram_numeric_selected = hstack([X_train_bigram_numeric_csr.getcol(i) for i in selected_feature_index[:1000]])

In [108]:

X_test_bigram_numeric_selected = hstack([X_test_bigram_numeric_csr.getcol(i) for i in selected_feature_index[:1000]])

In [100]:
test_scores = []
train_scores = []
max_features_list = [3,4,5  ]
for this_max_features in max_features_list:
    rf = RandomForestClassifier(n_jobs=-1, max_features=this_max_features, n_estimators=500)
    rf.fit(X_train_bigram_numeric_selected, y_train)
    tr = rf.score(X_train_bigram_numeric_selected, y_train)
    te = rf.score(X_test_bigram_numeric_selected, y_test)
    print("max features = {}, train score = {}, test score = {}\n".format(this_max_features,tr,te))
    train_scores.append(tr)
    test_scores.append(te)

max features = 3, train score = 1.0, test score = 0.8212800214678653

max features = 4, train score = 1.0, test score = 0.8274520327384945

max features = 5, train score = 1.0, test score = 0.8295988192674091



In [59]:
1394661 * 0.000001

1.394661

In [109]:
test_scores = []
train_scores = []
C_list = [2,5,10]
for this_C in C_list:
    lr = LogisticRegression(C= this_C, penalty = 'l2', random_state=0)
    lr.fit(X_train_bigram_numeric_selected, y_train)
    train_score = lr.score(X_train_bigram_numeric_selected, y_train)
    test_score = lr.score(X_test_bigram_numeric_selected, y_test)
    print("C = {}, train score = {}, test score = {}\n".format(this_C,train_score,test_score))
    train_scores.append(train_score)
    test_scores.append(test_score)



C = 2, train score = 0.870101986044015, test score = 0.8376492687508386

C = 5, train score = 0.8721595992127393, test score = 0.8372467462766671

C = 10, train score = 0.8730094829128645, test score = 0.8352341339058097



In [ ]:
vectorizer = CountVectorizer(decode_error="ignore",ngram_range=(1, 2))
X_train_title_bigram = vectorizer.fit_transform(Data_train.title)
X_test_title_bigram  = vectorizer.transform(Data_test.title)
X_train_abstract_bigram = vectorizer.fit_transform(Data_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(Data_test.abstract)
Data_train.references = Data_train.references.fillna("")
Data_test.references = Data_test.references.fillna("")
X_train_references_bigram = vectorizer.fit_transform(Data_train.references)
X_test_references_bigram  = vectorizer.transform(Data_test.references)

In [ ]:
from scipy.sparse import hstack
real_X_train_bigram_numeric = hstack([X_train_title_bigram, X_train_abstract_bigram, X_train_references_bigram, Data_train.iloc[:, [3,5]]])
real_X_test_bigram_numeric  = hstack([X_test_title_bigram, X_test_abstract_bigram, X_test_references_bigram, Data_test.iloc[:, [3,5]]])

In [28]:
lr = LogisticRegression(C= 1000, penalty = 'l1')
lr.fit(real_X_train_bigram_numeric, Data_train.iloc[:,6])

/Users/lvkunsheng/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
lr.score(real_X_train_bigram_numeric, Data_train.iloc[:,6])


1.0

In [34]:
prediction = pd.DataFrame(lr.predict(real_X_test_bigram_numeric), index = Data_test.test_id)
prediction.to_csv("prediction1029_v2.csv")

# Trying to get some new features

In [1]:
import numpy as np
import pandas as pd
Data_train = pd.read_csv("train.csv")
Data_test  = pd.read_csv("test.csv")

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

X_train, X_test, y_train, y_test = train_test_split(Data_train.iloc[:,1:6], Data_train.iloc[:,6],random_state = 0, stratify = Data_train.iloc[:,6])


X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, random_state = 0, stratify = y_train)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = TfidfVectorizer(decode_error="ignore",ngram_range=(1, 3))

X_train_title_bigram = vectorizer.fit_transform(X_train.title)
X_test_title_bigram  = vectorizer.transform(X_test.title)

X_train_abstract_bigram = vectorizer.fit_transform(X_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(X_test.abstract)

vectorizer = TfidfVectorizer(decode_error="ignore")
X_train.references = X_train.references.fillna("")
X_test.references = X_test.references.fillna("")
X_train_references_bigram = vectorizer.fit_transform(X_train.references)
X_test_references_bigram  = vectorizer.transform(X_test.references)
from scipy.sparse import coo_matrix, hstack
X_train_bigram_numeric = hstack([X_train_title_bigram, X_train_abstract_bigram, X_train_references_bigram, X_train.iloc[:, [2,4]]])
X_test_bigram_numeric  = hstack([X_test_title_bigram, X_test_abstract_bigram, X_test_references_bigram, X_test.iloc[:, [2,4]]])


In [9]:
test_scores = []
train_scores = []
C_list = np.linspace(0.1, 10, 20)
for this_C in C_list:
    lr = LogisticRegression(C= this_C, penalty = 'l2', random_state=0)
    lr.fit(X_train_bigram_numeric, y_train)
    tr = lr.score(X_train_bigram_numeric, y_train)
    te = lr.score(X_test_bigram_numeric, y_test)
    print("C = {},Training score is {}, testing score is {}\n".format(this_C, tr, te))
    train_scores.append(tr)
    test_scores.append(te)


C = 0.1,Training score is 0.7139470388262659, testing score is 0.6547698913189319

C = 0.6210526315789474,Training score is 0.7368938987296475, testing score is 0.6850932510398497

C = 1.142105263157895,Training score is 0.7368491680085884, testing score is 0.6850932510398497

C = 1.6631578947368424,Training score is 0.7368491680085884, testing score is 0.6850932510398497

C = 2.18421052631579,Training score is 0.7368491680085884, testing score is 0.6850932510398497

C = 2.7052631578947373,Training score is 0.7368044372875291, testing score is 0.6850932510398497

C = 3.2263157894736847,Training score is 0.7368044372875291, testing score is 0.6850932510398497

C = 3.747368421052632,Training score is 0.7367597065664698, testing score is 0.6850932510398497

C = 4.268421052631579,Training score is 0.7367597065664698, testing score is 0.6850932510398497

C = 4.7894736842105265,Training score is 0.7367597065664698, testing score is 0.6850932510398497

C = 5.310526315789474,Training score is 

In [ ]:
rf = RandomForestClassifier(n_jobs=-1, max_features=this_max_features, n_estimators=200, random_state = 0)
rf.fit(X_train_bigram_numeric, y_train)

In [12]:
from sklearn.ensemble import RandomForestClassifier
test_scores = {}
train_scores = {}
max_features_list = list(range(3,30,5))
# max_depth_list = list(range(50,75,5))

#for this_max_features in max_features_list:
rf = RandomForestClassifier(n_jobs=-1, max_features=3, max_depth= 1000,n_estimators=200, random_state = 0)
rf.fit(X_train_bigram_numeric, y_train)
tr = rf.score(X_train_bigram_numeric, y_train)
te = rf.score(X_test_bigram_numeric, y_test)
print("max features = {},train score = {}, test score = {}\n".format(this_max_features,tr,te))


max features = 28,train score = 0.7400697799248523, test score = 0.38964175499798737



In [ ]:
from sklearn.svm import SVC
test_scores = []
train_scores = []
C_list = list(range(1,20,2))
for this_C in C_list:
    svc = SVC(kernel="poly", C = this_C, gamma = "scale")
    svc.fit(X_train_bigram_numeric, y_train)
    tr = svc.score(X_train_bigram_numeric, y_train)
    te = svc.score(X_test_bigram_numeric, y_test)
    print("C = {}, train score = {}, test score = {}\n".format(this_C,tr,te))
    train_scores.append(tr)
    test_scores.append(te)

In [ ]:
from sklearn.svm import SVC
test_scores = []
train_scores = []
C_list = list(range(1,20,2))
for this_C in C_list:
    svc = SVC(kernel="linear", C = this_C, gamma = "scale")
    svc.fit(X_train_bigram_numeric, y_train)
    tr = svc.score(X_train_bigram_numeric, y_train)
    te = svc.score(X_test_bigram_numeric, y_test)
    print("C = {}, train score = {}, test score = {}\n".format(this_C,tr,te))
    train_scores.append(tr)
    test_scores.append(te)

In [7]:
vectorizer = TfidfVectorizer(decode_error="ignore",ngram_range=(1, 3))
X_train_title_bigram = vectorizer.fit_transform(Data_train.title)
X_test_title_bigram  = vectorizer.transform(Data_test.title)
X_train_abstract_bigram = vectorizer.fit_transform(Data_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(Data_test.abstract)

vectorizer = TfidfVectorizer(decode_error="ignore")
Data_train.references = Data_train.references.fillna("")
Data_test.references = Data_test.references.fillna("")
X_train_references_bigram = vectorizer.fit_transform(Data_train.references)
X_test_references_bigram  = vectorizer.transform(Data_test.references)
from scipy.sparse import hstack
real_X_train_bigram_numeric = hstack([X_train_title_bigram, X_train_abstract_bigram, X_train_references_bigram, Data_train.iloc[:, [3,5]]])
real_X_test_bigram_numeric  = hstack([X_test_title_bigram, X_test_abstract_bigram, X_test_references_bigram, Data_test.iloc[:, [3,5]]])

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=100, random_state=0)
clf.fit(X_train_bigram_numeric, y_train)
tr = clf.score(X_train_bigram_numeric, y_train)
te = clf.score(X_test_bigram_numeric, y_test)
print("train score = {}, test score = {}\n".format(tr,te))

# trash tfidf

In [5]:
import numpy as np
import pandas as pd
Data_train = pd.read_csv("train.csv")
Data_test  = pd.read_csv("test.csv")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

X_train, X_test, y_train, y_test = train_test_split(Data_train.iloc[:,1:6], Data_train.iloc[:,6],random_state = 0, stratify = Data_train.iloc[:,6])


X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, random_state = 0, stratify = y_train)
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(decode_error="ignore",ngram_range=(1, 2))

X_train_title_bigram = vectorizer.fit_transform(X_train.title)
X_test_title_bigram  = vectorizer.transform(X_test.title)

X_train_abstract_bigram = vectorizer.fit_transform(X_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(X_test.abstract)

vectorizer = CountVectorizer(decode_error="ignore")
X_train.references = X_train.references.fillna("")
X_test.references = X_test.references.fillna("")
X_train_references_bigram = vectorizer.fit_transform(X_train.references)
X_test_references_bigram  = vectorizer.transform(X_test.references)
from scipy.sparse import coo_matrix, hstack
X_train_bigram_numeric = hstack([X_train_title_bigram, X_train_abstract_bigram, X_train_references_bigram, X_train.iloc[:, [2,4]]])
X_test_bigram_numeric  = hstack([X_test_title_bigram, X_test_abstract_bigram, X_test_references_bigram, X_test.iloc[:, [2,4]]])


In [4]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),n_estimators=200, random_state=0)
clf.fit(X_train_bigram_numeric, y_train)
tr = clf.score(X_train_bigram_numeric, y_train)
te = clf.score(X_test_bigram_numeric, y_test)
print("train score = {}, test score = {}\n".format(tr,te))

train score = 0.8445160135981392, test score = 0.8139004427747216



In [5]:
scores = {}
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(X_train_bigram_numeric, y_train)
model.score(X_test_bigram_numeric,y_test)

NameError: name 'x_test' is not defined

In [19]:
from scipy.stats import uniform, randint
params = {
     "colsample_bytree": uniform(0.7, 0.3),
     "gamma": uniform(0, 0.5),
     "learning_rate": uniform(0.03, 0.3), # default 0.1 
     "max_depth": randint(2, 6), # default 3
     "n_estimators": randint(100, 150), # default 100
     "subsample": uniform(0.6, 0.4)
    }
params

{'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen at 0x22a4345a6d8>,
 'gamma': <scipy.stats._distn_infrastructure.rv_frozen at 0x22a4345ac50>,
 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen at 0x22a4345add8>,
 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen at 0x22a4345af60>,
 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen at 0x22a4345ada0>,
 'subsample': <scipy.stats._distn_infrastructure.rv_frozen at 0x22a434230b8>}

In [ ]:
test_scores = {}
train_scores = {}

import xgboost as xgb
import numpy as np
for i in range(100):
    colsample_bytree = np.random.uniform(0.7, 0.3)
    gamma         = np.random.uniform(0, 0.5)
    learning_rate = np.random.uniform(0.08, 0.3)
    max_depth     = np.random.randint(2, 6)
    n_estimators  = np.random.randint(100, 150)
    subsample     = np.random.uniform(0.6, 0.4)
    params = {
     "colsample_bytree": colsample_bytree,
     "gamma": gamma,
     "learning_rate": learning_rate, # default 0.1 
     "max_depth": max_depth, # default 3
     "n_estimators": n_estimators, # default 100
     "subsample": subsample
    }
    model=xgb.XGBClassifier(objective="multi:softprob",eval_metric = "merror",random_state=1,**params )
    model.fit(X_train_bigram_numeric, y_train)
    te = model.score(X_test_bigram_numeric,y_test)
    tr = model.score(X_train_bigram_numeric, y_train)
    test_scores[tuple(params.values())] = te
    train_scores[tuple(params.values())] = tr
    print("Params = {}, train score = {}, test score = {}\n\n".format(params, tr, te))

Params = {'colsample_bytree': 0.5649476632289705, 'gamma': 0.2542327337891345, 'learning_rate': 0.23418998463693103, 'max_depth': 4, 'n_estimators': 112, 'subsample': 0.5122059514323735}, train score = 0.9068259080336375, test score = 0.8603247014624983


Params = {'colsample_bytree': 0.6940527346556096, 'gamma': 0.48141981866791034, 'learning_rate': 0.14129983892119768, 'max_depth': 4, 'n_estimators': 108, 'subsample': 0.5275992768833908}, train score = 0.8776614779030238, test score = 0.8505299879243258


Params = {'colsample_bytree': 0.3308773376543079, 'gamma': 0.24262554278833465, 'learning_rate': 0.14904783635568786, 'max_depth': 5, 'n_estimators': 149, 'subsample': 0.43760200008140593}, train score = 0.9130434782608695, test score = 0.8644841003622702


Params = {'colsample_bytree': 0.3343265451473223, 'gamma': 0.2010554018708826, 'learning_rate': 0.2723581353036539, 'max_depth': 2, 'n_estimators': 146, 'subsample': 0.49848749761022787}, train score = 0.8706387546967257, test sc

Params = {'colsample_bytree': 0.38711087700349944, 'gamma': 0.2970084687931152, 'learning_rate': 0.10361862721911703, 'max_depth': 4, 'n_estimators': 127, 'subsample': 0.518785489471999}, train score = 0.8702809089282519, test score = 0.8442238024956393


Params = {'colsample_bytree': 0.32411005369767226, 'gamma': 0.07325119505848582, 'learning_rate': 0.2963620655792649, 'max_depth': 2, 'n_estimators': 122, 'subsample': 0.48772439705311665}, train score = 0.8665235283592772, test score = 0.8489198980276399


Params = {'colsample_bytree': 0.5947497820346047, 'gamma': 0.31121535314086496, 'learning_rate': 0.2430383100800883, 'max_depth': 5, 'n_estimators': 106, 'subsample': 0.5454172576672109}, train score = 0.9272231168366434, test score = 0.8650207969944989


Params = {'colsample_bytree': 0.5813885841263848, 'gamma': 0.27912674541614085, 'learning_rate': 0.17046765650291373, 'max_depth': 2, 'n_estimators': 126, 'subsample': 0.5485655027981216}, train score = 0.8458579352299159, test sc

In [6]:
import numpy as np
import pandas as pd
Data_train = pd.read_csv("train.csv")
Data_test  = pd.read_csv("test.csv")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MaxAbsScaler

X_train, X_test, y_train, y_test = train_test_split(Data_train.iloc[:,1:6], Data_train.iloc[:,6],random_state = 0, stratify = Data_train.iloc[:,6])


X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, random_state = 0, stratify = y_train)
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(decode_error="ignore",ngram_range=(1, 2))

X_train_title_bigram = vectorizer.fit_transform(X_train.title)
X_test_title_bigram  = vectorizer.transform(X_test.title)

X_train_abstract_bigram = vectorizer.fit_transform(X_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(X_test.abstract)

vectorizer = CountVectorizer(decode_error="ignore")
X_train.references = X_train.references.fillna("")
X_test.references = X_test.references.fillna("")
X_train_references_bigram = vectorizer.fit_transform(X_train.references)
X_test_references_bigram  = vectorizer.transform(X_test.references)
from scipy.sparse import coo_matrix, hstack
X_train_bigram_numeric = hstack([X_train_title_bigram, X_train_abstract_bigram, X_train_references_bigram, X_train.iloc[:, [2,4]]])
X_test_bigram_numeric  = hstack([X_test_title_bigram, X_test_abstract_bigram, X_test_references_bigram, X_test.iloc[:, [2,4]]])

scaler = MaxAbsScaler()
X_train_bigram_numeric_scaled = scaler.fit_transform(X_train_bigram_numeric)
X_test_bigram_numeric_scaled  = scaler.transform(X_test_bigram_numeric)


In [7]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = MultinomialNB()
estimators.append(('MulNB', model2))
# model2 = DecisionTreeClassifier()
# estimators.append(('cart', model2))
model3 = LinearSVC(C = 0.00838888888888889)
estimators.append(('svm', model3))
model4 = RandomForestClassifier(n_estimators = 300)
estimators.append(('rf', model4))
params = {'colsample_bytree': 0.47205931649328914, 'gamma': 0.2520474500722088, 'learning_rate': 0.2871749273323586, 'max_depth': 5, 'n_estimators': 144, 'subsample': 0.5515106522016212}
model5 = xgb.XGBClassifier(objective="multi:softprob",eval_metric = "merror",random_state=1,**params )
estimators.append(('xgb1', model5))
# create the ensemble model
ensemble = VotingClassifier(estimators, n_jobs = -1)

ensemble.fit(X_train_bigram_numeric_scaled, y_train)
tr = ensemble.score(X_train_bigram_numeric_scaled, y_train)
te = ensemble.score(X_test_bigram_numeric_scaled, y_test)
print("train score = {}, test score = {}\n".format(tr,te))

train score = 0.9998210771157631, test score = 0.8934657185026164



In [8]:
vectorizer = CountVectorizer(decode_error="ignore",ngram_range=(1, 2))
X_train_title_bigram = vectorizer.fit_transform(Data_train.title)
X_test_title_bigram  = vectorizer.transform(Data_test.title)
X_train_abstract_bigram = vectorizer.fit_transform(Data_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(Data_test.abstract)
Data_train.references = Data_train.references.fillna("")
Data_test.references = Data_test.references.fillna("")
vectorizer = CountVectorizer(decode_error="ignore")
X_train_references_bigram = vectorizer.fit_transform(Data_train.references)
X_test_references_bigram  = vectorizer.transform(Data_test.references)
from scipy.sparse import hstack
real_X_train_bigram_numeric = hstack([X_train_title_bigram, X_train_abstract_bigram, X_train_references_bigram, Data_train.iloc[:, [3,5]]])
real_X_test_bigram_numeric  = hstack([X_test_title_bigram, X_test_abstract_bigram, X_test_references_bigram, Data_test.iloc[:, [3,5]]])

scaler = MaxAbsScaler()
real_X_train_bigram_numeric_scaled = scaler.fit_transform(real_X_train_bigram_numeric)
real_X_test_bigram_numeric_scaled  = scaler.transform(real_X_test_bigram_numeric)

In [9]:
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:,6])
prediction = ensemble.predict(real_X_test_bigram_numeric_scaled)
prediction = pd.DataFrame(prediction, index = Data_test.test_id)
prediction.to_csv("prediction1102_v3.csv")

## new features

In [10]:
import numpy as np
import pandas as pd
Data_train = pd.read_csv("train.csv")
Data_test  = pd.read_csv("test.csv")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MaxAbsScaler

X_train, X_test, y_train, y_test = train_test_split(Data_train.iloc[:,1:6], Data_train.iloc[:,6],random_state = 0, stratify = Data_train.iloc[:,6])


X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, random_state = 0, stratify = y_train)
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(decode_error="ignore",ngram_range=(1, 2))

X_train_title_bigram = vectorizer.fit_transform(X_train.title)
X_test_title_bigram  = vectorizer.transform(X_test.title)

X_train_abstract_bigram = vectorizer.fit_transform(X_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(X_test.abstract)

vectorizer = CountVectorizer(decode_error="ignore")
X_train.references = X_train.references.fillna("")
X_test.references = X_test.references.fillna("")
X_train_references_bigram = vectorizer.fit_transform(X_train.references)
X_test_references_bigram  = vectorizer.transform(X_test.references)

from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()
X_train_year_onehot = encode.fit_transform(np.array(X_train.iloc[:,2]).reshape(-1, 1))
X_test_year_onehot = encode.transform(np.array(X_test.iloc[:,2]).reshape(-1, 1))
from scipy.sparse import coo_matrix, hstack
X_train_bigram_numeric = hstack([X_train_title_bigram, 
                                 X_train_abstract_bigram, 
                                 X_train_references_bigram, 
                                 X_train_year_onehot, 
                                 np.array(X_train.iloc[:, 4]).reshape(-1, 1)])
X_test_bigram_numeric  = hstack([X_test_title_bigram, 
                                 X_test_abstract_bigram, 
                                 X_test_references_bigram, 
                                 X_test_year_onehot, 
                                 np.array(X_test.iloc[:, 4]).reshape(-1, 1)])

scaler = MaxAbsScaler()
X_train_bigram_numeric_scaled = scaler.fit_transform(X_train_bigram_numeric)
X_test_bigram_numeric_scaled  = scaler.transform(X_test_bigram_numeric)


D:\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [ ]:
test_scores = {}
train_scores = {}

import xgboost as xgb
import numpy as np
for i in range(100):
    colsample_bytree = np.random.uniform(0.7, 0.3)
    gamma         = np.random.uniform(0, 0.5)
    learning_rate = np.random.uniform(0.08, 0.3)
    max_depth     = np.random.randint(2, 6)
    n_estimators  = np.random.randint(100, 150)
    subsample     = np.random.uniform(0.6, 0.4)
    params = {
     "colsample_bytree": colsample_bytree,
     "gamma": gamma,
     "learning_rate": learning_rate, # default 0.1 
     "max_depth": max_depth, # default 3
     "n_estimators": n_estimators, # default 100
     "subsample": subsample
    }
    model=xgb.XGBClassifier(objective="multi:softprob",eval_metric = "merror",random_state=1997,**params )
    model.fit(X_train_bigram_numeric_scaled, y_train)
    te = model.score(X_test_bigram_numeric_scaled,y_test)
    tr = model.score(X_train_bigram_numeric_scaled, y_train)
    test_scores[tuple(params.values())] = te
    train_scores[tuple(params.values())] = tr
    print("Params = {}, train score = {}, test score = {}\n\n".format(params, tr, te))

Params = {'colsample_bytree': 0.49415695293611894, 'gamma': 0.40494950353418324, 'learning_rate': 0.16336584941488316, 'max_depth': 2, 'n_estimators': 115, 'subsample': 0.4431513819908308}, train score = 0.8377169439971373, test score = 0.8285254260029519


Params = {'colsample_bytree': 0.5239025896736734, 'gamma': 0.2983399602643831, 'learning_rate': 0.21350179889275844, 'max_depth': 5, 'n_estimators': 142, 'subsample': 0.4525243029148969}, train score = 0.9317409196636249, test score = 0.8635448812558701


Params = {'colsample_bytree': 0.5919179976548663, 'gamma': 0.07466046036833285, 'learning_rate': 0.23109068962853985, 'max_depth': 3, 'n_estimators': 144, 'subsample': 0.5804171042985782}, train score = 0.8941671139738773, test score = 0.8585804374077552


Params = {'colsample_bytree': 0.5810435725142817, 'gamma': 0.05878923298268307, 'learning_rate': 0.12474334049375965, 'max_depth': 2, 'n_estimators': 108, 'subsample': 0.5780395348267672}, train score = 0.8213007693684022, test s

Params = {'colsample_bytree': 0.4673720171973651, 'gamma': 0.015265040656822937, 'learning_rate': 0.19437910230823707, 'max_depth': 5, 'n_estimators': 111, 'subsample': 0.4154449708018563}, train score = 0.9116120951869744, test score = 0.8580437407755266


Params = {'colsample_bytree': 0.48947291923779734, 'gamma': 0.39493905384137173, 'learning_rate': 0.11155628743708404, 'max_depth': 4, 'n_estimators': 109, 'subsample': 0.5474925259400785}, train score = 0.8612005725532296, test score = 0.8356366563799812


Params = {'colsample_bytree': 0.44027932682065674, 'gamma': 0.42783790467091704, 'learning_rate': 0.18529528617422542, 'max_depth': 4, 'n_estimators': 129, 'subsample': 0.5976504972716395}, train score = 0.8989085704061549, test score = 0.8583120890916409


Params = {'colsample_bytree': 0.4661797542514529, 'gamma': 0.18517650139399844, 'learning_rate': 0.22115666372676362, 'max_depth': 3, 'n_estimators': 106, 'subsample': 0.5926853896463291}, train score = 0.8758722490606549, tes

Params = {'colsample_bytree': 0.5299677815756025, 'gamma': 0.019910690002598763, 'learning_rate': 0.14519599162090369, 'max_depth': 2, 'n_estimators': 138, 'subsample': 0.42248786092695745}, train score = 0.8421900161030595, test score = 0.8321481282704951


Params = {'colsample_bytree': 0.5909903549675323, 'gamma': 0.29852809661171925, 'learning_rate': 0.09897516959465198, 'max_depth': 2, 'n_estimators': 122, 'subsample': 0.5536205953726864}, train score = 0.8161567364465915, test score = 0.8109486112974641


Params = {'colsample_bytree': 0.31627065012396266, 'gamma': 0.3151098460505343, 'learning_rate': 0.2192439741390398, 'max_depth': 2, 'n_estimators': 124, 'subsample': 0.44086343288257135}, train score = 0.853283234925747, test score = 0.8411377968603247


Params = {'colsample_bytree': 0.5854012895805467, 'gamma': 0.4693077232916942, 'learning_rate': 0.17982887811854487, 'max_depth': 3, 'n_estimators': 115, 'subsample': 0.5321720386897326}, train score = 0.8695652173913043, test s

## itf + GSD + ensemble

In [4]:
import numpy as np
import pandas as pd
Data_train = pd.read_csv("train.csv")
Data_test  = pd.read_csv("test.csv")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MaxAbsScaler

X_train, X_test, y_train, y_test = train_test_split(Data_train.iloc[:,1:6], Data_train.iloc[:,6],random_state = 0, stratify = Data_train.iloc[:,6])


X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, random_state = 0, stratify = y_train)
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = TfidfVectorizer(decode_error="ignore",ngram_range=(1, 3))

X_train_title_bigram = vectorizer.fit_transform(X_train.title)
X_test_title_bigram  = vectorizer.transform(X_test.title)

X_train_abstract_bigram = vectorizer.fit_transform(X_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(X_test.abstract)

vectorizer = TfidfVectorizer(decode_error="ignore",ngram_range=(1, 2))
X_train.references = X_train.references.fillna("")
X_test.references = X_test.references.fillna("")
X_train_references_bigram = vectorizer.fit_transform(X_train.references)
X_test_references_bigram  = vectorizer.transform(X_test.references)

from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()
X_train_year_onehot = encode.fit_transform(np.array(X_train.iloc[:,2]).reshape(-1,1))
X_test_year_onehot = encode.transform(np.array(X_test.iloc[:,2]).reshape(-1,1))

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)

X_train_year = poly.fit_transform(np.array(X_train.iloc[:,2]).reshape(-1,1))
X_test_year  = poly.transform(np.array(X_test.iloc[:,2]).reshape(-1,1))

X_train_n_citation = poly.fit_transform(np.array(X_train.iloc[:,4]).reshape(-1,1))
X_test_n_citation  = poly.transform(np.array(X_test.iloc[:,4]).reshape(-1,1))
from scipy.sparse import coo_matrix, hstack
X_train_bigram_numeric = hstack([X_train_title_bigram, 
                                 X_train_abstract_bigram, 
                                 X_train_references_bigram, 
                                 X_train_year_onehot, 
                                 X_train_year,
                                 X_train_n_citation])
X_test_bigram_numeric  = hstack([X_test_title_bigram, 
                                 X_test_abstract_bigram, 
                                 X_test_references_bigram, 
                                 X_test_year_onehot, 
                                 X_test_year,
                                 X_test_n_citation])

scaler = MaxAbsScaler()
X_train_bigram_numeric_scaled = scaler.fit_transform(X_train_bigram_numeric)
X_test_bigram_numeric_scaled  = scaler.transform(X_test_bigram_numeric)

D:\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [15]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier


# import xgboost as xgb
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
#model2 = MultinomialNB()
#estimators.append(('MulNB', model2))
model41 = SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42, max_iter=5, tol=None)
estimators.append(('SGD1', model41))
model42 = SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-2, random_state=42, max_iter=5, tol=None)
estimators.append(('SGD2', model42))
model43 = SGDClassifier(loss='hinge', penalty='l2',
                        alpha=0.0025, random_state=40, max_iter=5, tol=None)
estimators.append(('SGD3', model43))

model3 = LinearSVC(C = 0.01)
estimators.append(('svm', model3))
model3_1 = LinearSVC(C = 1)
estimators.append(('svm1', model3_1))
# model4 = RandomForestClassifier(n_estimators = 300)
# estimators.append(('rf', model4))
# params = {'colsample_bytree': 0.47205931649328914, 'gamma': 0.2520474500722088, 'learning_rate': 0.2871749273323586, 'max_depth': 5, 'n_estimators': 144, 'subsample': 0.5515106522016212}
# model5 = xgb.XGBClassifier(objective="multi:softprob",eval_metric = "merror",random_state=1,**params )
# estimators.append(('xgb1', model5))
# create the ensemble model
ensemble = VotingClassifier(estimators, n_jobs = -1)

ensemble.fit(X_train_bigram_numeric_scaled, y_train)
tr = ensemble.score(X_train_bigram_numeric_scaled, y_train)
te = ensemble.score(X_test_bigram_numeric_scaled, y_test)
print("train score = {}, test score = {}\n".format(tr,te))

train score = 1.0, test score = 0.894002415134845



In [5]:
from sklearn.neural_network import MLPClassifier
#scores = []
#for this_size in range(19,36):
model5 = MLPClassifier(hidden_layer_sizes=(50), 
                       random_state = 0, 
                       verbose = True) 
                       #n_iter_no_change=5,
                       #learning_rate_init = 0.01,
                       #early_stopping=True)
model5.fit(X_train_bigram_numeric_scaled, y_train)
tr = model5.score(X_train_bigram_numeric_scaled, y_train)
te = model5.score(X_test_bigram_numeric_scaled, y_test)
#scores.append(te)
print("size = {},train score = {}, test score = {}\n".format(this_size, tr,te))

#n_iter_no_change=10, 0.901
#n_iter_no_change=5, 0.9001
#n_iter_no_change=3， 0.899
# 2， 0.899
# 1， 0.899

# 1，0.01，

# 4,8:0.88
# 6,8:0.89
# 12,8:0.898
# 4,12,8: 0.86
# 12,12,8: 0.88
# 100

Iteration 1, loss = 0.53564703
Iteration 2, loss = 0.00461451
Iteration 3, loss = 0.00308172
Iteration 4, loss = 0.00269152
Iteration 5, loss = 0.00242104
Iteration 6, loss = 0.00228914
Iteration 7, loss = 0.00219043
Iteration 8, loss = 0.00209872
Iteration 9, loss = 0.00202406
Iteration 10, loss = 0.00195738
Iteration 11, loss = 0.00189530
Iteration 12, loss = 0.00183761
Iteration 13, loss = 0.00178106
Iteration 14, loss = 0.00172807
Iteration 15, loss = 0.00167686
Iteration 16, loss = 0.00162814
Iteration 17, loss = 0.00158072
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


NameError: name 'this_size' is not defined

In [6]:
print("train score = {}, test score = {}\n".format(tr,te))

train score = 1.0, test score = 0.8917214544478733



In [24]:
from joblib import Parallel, delayed
from sklearn.neural_network import MLPClassifier
def myfun(arg):
    model5 = MLPClassifier(hidden_layer_sizes=(arg), random_state = 0, verbose = True)
    model5.fit(X_train_bigram_numeric_scaled, y_train)
    tr = model5.score(X_train_bigram_numeric_scaled, y_train)
    te = model5.score(X_test_bigram_numeric_scaled, y_test)
    print("size = {},train score = {}, test score = {}\n".format(arg, tr,te))
    return te

#results = Parallel(n_jobs=3, verbose=True, backend="threading")(delayed(myfun)(i) for i in range(16,100))

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


Iteration 1, loss = 0.75377184
Iteration 1, loss = 0.71557782
Iteration 1, loss = 0.77978001
Iteration 2, loss = 0.05753910
Iteration 2, loss = 0.07186726
Iteration 2, loss = 0.04287427
Iteration 3, loss = 0.00895958
Iteration 3, loss = 0.00901511
Iteration 3, loss = 0.00806472
Iteration 4, loss = 0.00391338
Iteration 4, loss = 0.00331034
Iteration 4, loss = 0.00355714
Iteration 5, loss = 0.00263973
Iteration 5, loss = 0.00208084
Iteration 5, loss = 0.00230595
Iteration 6, loss = 0.00202835
Iteration 6, loss = 0.00174534
Iteration 6, loss = 0.00159948
Iteration 7, loss = 0.00169606
Iteration 7, loss = 0.00148458
Iteration 7, loss = 0.00134455
Iteration 8, loss = 0.00148589
Iteration 8, loss = 0.00132293
Iteration 8, loss = 0.00118749
Iteration 9, loss = 0.00134156
Iteration 9, loss = 0.00121228
Iteration 9, loss = 0.00108085
Iteration 10, loss = 0.00123603
Iteration 10, loss = 0.00113094
Iteration 10, loss = 0.00100346
Iteration 11, loss = 0.00115529
Iteration 11, loss = 0.00094426
Ite

KeyboardInterrupt: 

(5)  : 0.897
(8)m : 0.897
(8)  ： 0.898
（8）0.0001：0.896 
(10) : default 0.89// random  0.90205
（11）：0.8996
（20）：0.9001
(2, 5): not default 0.73
(2,5) : default 0.81
(2,10) : 0.76
(8,4)m : 0.86
(8,6) : 0.89
（10，10）：0.8988
(10,2) : 0.84
（3，3）m： 0.83

above 0.9: 10，11，14，15，16，18,20，22和20一样，24，26，28，29，31(高一点0.901)，
32，33，

In [7]:
vectorizer = TfidfVectorizer(decode_error="ignore",ngram_range=(1, 2))
X_train_title_bigram = vectorizer.fit_transform(Data_train.title)
X_test_title_bigram  = vectorizer.transform(Data_test.title)
X_train_abstract_bigram = vectorizer.fit_transform(Data_train.abstract)
X_test_abstract_bigram  = vectorizer.transform(Data_test.abstract)
Data_train.references = Data_train.references.fillna("")
Data_test.references = Data_test.references.fillna("")


X_train_references_bigram = vectorizer.fit_transform(Data_train.references)
X_test_references_bigram  = vectorizer.transform(Data_test.references)

from sklearn.preprocessing import OneHotEncoder
encode = OneHotEncoder()
X_train_year_onehot = encode.fit_transform(np.array(Data_train.year).reshape(-1,1))
X_test_year_onehot = encode.transform(np.array(Data_test.year).reshape(-1,1))

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)

X_train_year = poly.fit_transform(np.array(Data_train.year).reshape(-1,1))
X_test_year  = poly.transform(np.array(Data_test.year).reshape(-1,1))

X_train_n_citation = poly.fit_transform(np.array(Data_train.n_citation).reshape(-1,1))
X_test_n_citation  = poly.transform(np.array(Data_test.n_citation).reshape(-1,1))


from scipy.sparse import hstack
real_X_train_bigram_numeric = hstack([X_train_title_bigram, 
                                      X_train_abstract_bigram, 
                                      X_train_references_bigram, 
                                      X_train_year_onehot,
                                      X_train_year,
                                      X_train_n_citation])
real_X_test_bigram_numeric  = hstack([X_test_title_bigram, 
                                      X_test_abstract_bigram, 
                                      X_test_references_bigram, 
                                      X_test_year_onehot,
                                      X_test_year,
                                      X_test_n_citation])


scaler = MaxAbsScaler()
real_X_train_bigram_numeric_scaled = scaler.fit_transform(real_X_train_bigram_numeric)
real_X_test_bigram_numeric_scaled  = scaler.transform(real_X_test_bigram_numeric)

D:\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
from sklearn.neural_network import MLPClassifier
model5 = MLPClassifier(hidden_layer_sizes=(10), 
                       verbose = True,random_state = 0) 
model5.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])



Iteration 1, loss = 0.65204463
Iteration 2, loss = 0.03881700
Iteration 3, loss = 0.00813349
Iteration 4, loss = 0.00473853
Iteration 5, loss = 0.00335501
Iteration 6, loss = 0.00254591
Iteration 7, loss = 0.00207329
Iteration 8, loss = 0.00176021
Iteration 9, loss = 0.00153931
Iteration 10, loss = 0.00137764
Iteration 11, loss = 0.00125555
Iteration 12, loss = 0.00116023
Iteration 13, loss = 0.00108419
Iteration 14, loss = 0.00102176
Iteration 15, loss = 0.00097010
Iteration 16, loss = 0.00092610
Iteration 17, loss = 0.00088823
Iteration 18, loss = 0.00085523
Iteration 19, loss = 0.00082577
Iteration 20, loss = 0.00079938
Iteration 21, loss = 0.00077540
Iteration 22, loss = 0.00075341
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


AttributeError: 'DataFrame' object has no attribute 'tocsv'

In [6]:
prediction = pd.DataFrame(model5.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1103_v1_nn10.csv")

In [13]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
# import xgboost as xgb
estimators = []
#model1 = LogisticRegression()
#estimators.append(('logistic', model1))
# model2 = MultinomialNB()
# estimators.append(('MulNB', model2))
# model2 = DecisionTreeClassifier()
# estimators.append(('cart', model2))
# model3 = LinearSVC(C = 0.012345444444444445)
# estimators.append(('svm', model3))
#model2 = LinearSVC(C = 1)
#estimators.append(('svm', model2))
#model3 = SGDClassifier(loss='hinge', penalty='l2',
                        #alpha=1e-2, random_state=42, max_iter=5, tol=None)
#estimators.append(('SGD', model3))
from sklearn.neural_network import MLPClassifier
model4_1 = MLPClassifier(hidden_layer_sizes=(10), 
                       verbose = True,random_state = 0)
estimators.append(('NN1', model4_1))
model4_2 = MLPClassifier(hidden_layer_sizes=(11), 
                       verbose = True,random_state = 0)
estimators.append(('NN2', model4_2))
model4_3 = MLPClassifier(hidden_layer_sizes=(20), 
                       verbose = True,random_state = 0)
estimators.append(('NN3', model4_3))
# create the ensemble model
ensemble = VotingClassifier(estimators, n_jobs = -1)

ensemble.fit(X_train_bigram_numeric_scaled, y_train)
tr = ensemble.score(X_train_bigram_numeric_scaled, y_train)
te = ensemble.score(X_test_bigram_numeric_scaled, y_test)
print("train score = {}, test score = {}\n".format(tr,te))

train score = 1.0, test score = 0.9001744264054743



In [14]:

ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1104_v1_ensemble.csv")


In [49]:
model5 = MLPClassifier(#hidden_layer_sizes=(12,12,8), 
                       random_state = 0, 
                       verbose = True) 
                       #n_iter_no_change=5,
                       #learning_rate_init = 0.01,
                       #early_stopping=True)
model5.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(model5.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1107_v1.csv")

Iteration 1, loss = 0.44423794
Iteration 2, loss = 0.00665065
Iteration 3, loss = 0.00346165
Iteration 4, loss = 0.00301107
Iteration 5, loss = 0.00271769
Iteration 6, loss = 0.00245250
Iteration 7, loss = 0.00230934
Iteration 8, loss = 0.00214535
Iteration 9, loss = 0.00201858
Iteration 10, loss = 0.00191537
Iteration 11, loss = 0.00182184
Iteration 12, loss = 0.00173481
Iteration 13, loss = 0.00164950
Iteration 14, loss = 0.00157112
Iteration 15, loss = 0.00149687
Iteration 16, loss = 0.00142503
Iteration 17, loss = 0.00135985
Iteration 18, loss = 0.00129396
Iteration 19, loss = 0.00123211
Iteration 20, loss = 0.00117263
Iteration 21, loss = 0.00111642
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


In [22]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
# import xgboost as xgb
estimators = []
#model1 = LogisticRegression()
#estimators.append(('logistic', model1))
# model2 = MultinomialNB()
# estimators.append(('MulNB', model2))
# model2 = DecisionTreeClassifier()
# estimators.append(('cart', model2))
# model3 = LinearSVC(C = 0.012345444444444445)
# estimators.append(('svm', model3))
#model2 = LinearSVC(C = 1)
#estimators.append(('svm', model2))
#model3 = SGDClassifier(loss='hinge', penalty='l2',
                        #alpha=1e-2, random_state=42, max_iter=5, tol=None)
#estimators.append(('SGD', model3))
from sklearn.neural_network import MLPClassifier
estimators = []
model1 = MLPClassifier(hidden_layer_sizes=(10), 
                       verbose = True,random_state = 0)
estimators.append(('NN10', model1))
model2 = MLPClassifier(hidden_layer_sizes=(11), 
                       verbose = True,random_state = 0)
estimators.append(('NN11', model2))
model3 = MLPClassifier(hidden_layer_sizes=(14), 
                       verbose = True,random_state = 0)
estimators.append(('NN14', model3))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_10_11_14.csv")

estimators = []
model4 = MLPClassifier(hidden_layer_sizes=(15), 
                       verbose = True,random_state = 0)
estimators.append(('NN15', model4))
model5 = MLPClassifier(hidden_layer_sizes=(16), 
                       verbose = True,random_state = 0)
estimators.append(('NN16', model5))
model6 = MLPClassifier(hidden_layer_sizes=(18), 
                       verbose = True,random_state = 0)
estimators.append(('NN18', model6))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_15_16_18.csv")

estimators = []
model7 = MLPClassifier(hidden_layer_sizes=(24), 
                       verbose = True,random_state = 0)
estimators.append(('NN24', model7))
model8 = MLPClassifier(hidden_layer_sizes=(26), 
                       verbose = True,random_state = 0)
estimators.append(('NN26', model8))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_24_26.csv")

estimators = []
model9 = MLPClassifier(hidden_layer_sizes=(28), 
                       verbose = True,random_state = 0)
estimators.append(('NN28', model9))
model10 = MLPClassifier(hidden_layer_sizes=(29), 
                       verbose = True,random_state = 0)
estimators.append(('NN29', model10))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_28_29.csv")

estimators = []
model11 = MLPClassifier(hidden_layer_sizes=(31), 
                       verbose = True,random_state = 0)
estimators.append(('NN31', model11))
model12 = MLPClassifier(hidden_layer_sizes=(32), 
                       verbose = True,random_state = 0)
estimators.append(('NN32', model12))
ensemble = VotingClassifier(estimators, n_jobs =1)
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_31_32.csv")


model13 = MLPClassifier(hidden_layer_sizes=(33), 
                       verbose = True,random_state = 0)
estimators.append(('NN33', model13))
model13.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(model13.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble_33.csv")

# create the ensemble model
# ensemble = VotingClassifier(estimators, n_jobs =1)

#ensemble.fit(X_train_bigram_numeric_scaled, y_train)
#tr = ensemble.score(X_train_bigram_numeric_scaled, y_train)
#te = ensemble.score(X_test_bigram_numeric_scaled, y_test)
#print("train score = {}, test score = {}\n".format(tr,te))

Iteration 1, loss = 0.65204463
Iteration 2, loss = 0.03881700
Iteration 3, loss = 0.00813349
Iteration 4, loss = 0.00473853
Iteration 5, loss = 0.00335501
Iteration 6, loss = 0.00254591
Iteration 7, loss = 0.00207329
Iteration 8, loss = 0.00176021
Iteration 9, loss = 0.00153931
Iteration 10, loss = 0.00137764
Iteration 11, loss = 0.00125555
Iteration 12, loss = 0.00116023
Iteration 13, loss = 0.00108419
Iteration 14, loss = 0.00102176
Iteration 15, loss = 0.00097010
Iteration 16, loss = 0.00092610
Iteration 17, loss = 0.00088823
Iteration 18, loss = 0.00085523
Iteration 19, loss = 0.00082577
Iteration 20, loss = 0.00079938
Iteration 21, loss = 0.00077540
Iteration 22, loss = 0.00075341
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.70694764
Iteration 2, loss = 0.04636713
Iteration 3, loss = 0.00950413
Iteration 4, loss = 0.00543616
Iteration 5, loss = 0.00370199
Iteration 6, loss = 0.00279765
Iteration 7, loss = 0.0022589

Iteration 11, loss = 0.00118119
Iteration 12, loss = 0.00113411
Iteration 13, loss = 0.00109081
Iteration 14, loss = 0.00105061
Iteration 15, loss = 0.00101270
Iteration 16, loss = 0.00097706
Iteration 17, loss = 0.00094207
Iteration 18, loss = 0.00090918
Iteration 19, loss = 0.00087761
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.54023085
Iteration 2, loss = 0.01298664
Iteration 3, loss = 0.00341987
Iteration 4, loss = 0.00231356
Iteration 5, loss = 0.00178126
Iteration 6, loss = 0.00161206
Iteration 7, loss = 0.00149707
Iteration 8, loss = 0.00141286
Iteration 9, loss = 0.00134321
Iteration 10, loss = 0.00128341
Iteration 11, loss = 0.00123056
Iteration 12, loss = 0.00118285
Iteration 13, loss = 0.00113893
Iteration 14, loss = 0.00109729
Iteration 15, loss = 0.00105826
Iteration 16, loss = 0.00102155
Iteration 17, loss = 0.00098616
Iteration 18, loss = 0.00095210
Training loss did not improve more than tol=0.000100 fo

In [23]:
import os
os.listdir()

['.ipynb_checkpoints',
 'kaggle_project.ipynb',
 'prediction1029.csv',
 'prediction1102_v3.csv',
 'prediction1103_v1_nn10.csv',
 'prediction1103_v2_ensemble.csv',
 'prediction1104_v1_ensemble.csv',
 'prediction1107_v1.csv',
 'prediction1109_v1_nnensemble_10_11_14.csv',
 'prediction1109_v1_nnensemble_15_16_18.csv',
 'prediction1109_v1_nnensemble_24_26.csv',
 'prediction1109_v1_nnensemble_28_29.csv',
 'prediction1109_v1_nnensemble_31_32.csv',
 'prediction1109_v1_nnensemble_33.csv',
 'test.csv',
 'train.csv',
 'X_test.npz',
 'X_train.npz',
 'X_train_train.npz',
 'X_valid.npz']

In [25]:
import pandas as pd
import os
import re

from scipy import stats

df = pd.DataFrame()
for file in os.listdir():
    r = re.compile(r'prediction1109')
    temp = r.search(file)
    if temp is not None:
        df['{}'.format(file)] = pd.read_csv(file).iloc[:,1]
    
output = df.apply(lambda x: stats.mode(x)[0][0], axis = 1)

output.name = 'Category'
output.to_csv('prediction_13NN_ensemble.csv')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [21]:
ensemble.fit(real_X_train_bigram_numeric_scaled, Data_train.iloc[:, 6])
prediction = pd.DataFrame(ensemble.predict(real_X_test_bigram_numeric_scaled), index = Data_test.test_id)
prediction.to_csv("prediction1109_v1_nnensemble.csv")

Iteration 1, loss = 0.65204463
Iteration 2, loss = 0.03881700
Iteration 3, loss = 0.00813349
Iteration 4, loss = 0.00473853
Iteration 5, loss = 0.00335501
Iteration 6, loss = 0.00254591
Iteration 7, loss = 0.00207329
Iteration 8, loss = 0.00176021
Iteration 9, loss = 0.00153931
Iteration 10, loss = 0.00137764
Iteration 11, loss = 0.00125555
Iteration 12, loss = 0.00116023
Iteration 13, loss = 0.00108419
Iteration 14, loss = 0.00102176
Iteration 15, loss = 0.00097010
Iteration 16, loss = 0.00092610
Iteration 17, loss = 0.00088823
Iteration 18, loss = 0.00085523
Iteration 19, loss = 0.00082577
Iteration 20, loss = 0.00079938
Iteration 21, loss = 0.00077540
Iteration 22, loss = 0.00075341
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.70694764
Iteration 2, loss = 0.04636713
Iteration 3, loss = 0.00950413
Iteration 4, loss = 0.00543616
Iteration 5, loss = 0.00370199
Iteration 6, loss = 0.00279765
Iteration 7, loss = 0.0022589

MemoryError: 

In [13]:
ensemble

VotingClassifier(estimators=[('NN10',
                              MLPClassifier(activation='relu', alpha=0.0001,
                                            batch_size='auto', beta_1=0.9,
                                            beta_2=0.999, early_stopping=False,
                                            epsilon=1e-08,
                                            hidden_layer_sizes=10,
                                            learning_rate='constant',
                                            learning_rate_init=0.001,
                                            max_iter=200, momentum=0.9,
                                            n_iter_no_change=10,
                                            nesterovs_momentum=True,
                                            power_t=0.5, random_state=0,
                                            shuffle=True, solver='a...
                                            beta_2=0.999, early_stopping=False,
                                  